In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
from librep.base.estimator import Estimator
%load_ext autoreload
%autoreload 2
from utils.standartized_balanced import StandardizedBalancedDataset
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_data(dataset_name,sensors,normalize_data):    
    working_directory=f"result/{dataset_name}/"
    data_folder = f"../data/data/standartized_balanced/{dataset_name}/"
    dataset = StandardizedBalancedDataset(data_folder, sensors=sensors)
    X_train, y_train,X_test, y_test,X_val, y_val = dataset.get_all_data(normalize_data=normalize_data, resize_data=True)
    return X_train, y_train,X_test, y_test,X_val, y_val
        #print(f"shape: X_train {X_train.shape} --- X_test {X_test.shape} --- X_test {X_val.shape}")

/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from models.linear_model import LinearModel
from models.simclr_head import SimCLRHead
from simclr import SimCLR
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Estimator(Estimator):
    def __init__(self,trained_simclr_model,input_shape,
                batch_size_head,transform_funcs,temperature_head,epochs_head,
                 save_model=False,verbose=0,total_epochs=50,
                 batch_size=32,lr=0.001,classificator='full'):
       
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #self.loaded_simclr_model=trained_simclr_model
        self.batch_size=batch_size
        self.lr=lr
        self.total_epochs=total_epochs
        self.verbose=verbose
        self.classificador=classificator
        if trained_simclr_model==None:
            simclr_head = SimCLRHead(input_shape).to(device)
            self.simclr = SimCLR(model=simclr_head,batch_size=batch_size_head,
                                        transform_funcs=transform_funcs,
                                        temperature=temperature_head, epochs=epochs_head,
                                        is_transform_function_vectorized=True,
                                        verbose=verbose,device=device)
        else:
            self.simclr=trained_simclr_model
            

        
    def fit(self,X_train,y_train,X_val,y_val):
 
        
        trained_simclr_model,epoch_wise_loss = self.simclr.fit(X_train)


        # Define the number of classes for your downstream task
        num_classes = len(y_train[1])
        input_shape = X_train[1].shape  # Define your input shape        
        
        
        train_labels_int = torch.argmax(torch.tensor(y_train), dim=1)
        val_labels_int = torch.argmax(torch.tensor(y_val), dim=1)
        train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), train_labels_int)
        val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), val_labels_int)
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)
        if self.classificador=='full':            
            evaluation_model = LinearModel(trained_simclr_model, num_classes, ).to(self.device)
        self.best_model=evaluation_model


        

        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(evaluation_model.parameters(), self.lr)

        best_accuracy = 0.0
        best_model_path = "best_linear_evaluation_model_est.pth"

        for epoch in range(self.total_epochs):
            evaluation_model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = evaluation_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            if(self.verbose):
                print(f"Epoch [{epoch+1}/{self.total_epochs}] - Loss: {running_loss / len(train_loader)}")

            # Evaluate the model on the val set
            evaluation_model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = evaluation_model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            val_accuracy = correct / total
            if(self.verbose):
                print(f"Val Accuracy: {val_accuracy:.4f}")

            # Save the model if it's the best model so far
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                torch.save(evaluation_model.state_dict(), best_model_path)
                print(f"Best model saved with val accuracy: {best_accuracy:.4f}")

        print("Training finished.")
        # Load the saved model state dict
        self.best_model.load_state_dict(torch.load(best_model_path))
        return self
    
    
       
    def predict(self, input_data):
        """
        Predict class labels for input data.

        Args:
            input_data (numpy.ndarray or torch.Tensor): Input data for prediction.

        Returns:
            numpy.ndarray: Predicted class labels.
        """
        # Ensure input_data is a torch.Tensor
        if not isinstance(input_data, torch.Tensor):
            input_data = torch.tensor(input_data, dtype=torch.float32)

        # Move input data to the device (GPU or CPU)
        input_data = input_data.to(self.device)

        # Set the model to evaluation mode
        self.best_model.eval()

        # Perform predictions
        with torch.no_grad():
            outputs = self.best_model(input_data)
            predicted_classes = torch.argmax(outputs, dim=1)

        # Convert the PyTorch tensor to a NumPy array
        predicted_classes_np = predicted_classes.cpu().numpy()

        return predicted_classes_np


